**Imports**

In [ ]:
import os
import re
from pathlib import Path
from typing import Dict, List, Tuple
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from torchvision.models import ResNet18_Weights


In [ ]:
if torch.cuda.is_available():
    device_name = "cuda"
elif torch.backends.mps.is_available():
    device_name = "mps"
else:
    device_name = "cpu"
    
print(f'Using device: {device_name}')
device = torch.device(device_name)

**Load and Prepare Dataset**

In [ ]:
# Load labels
labels = pd.read_csv("../data/labels.csv")

image_paths = []
pattern_labels = []
division_labels = []

for idx, row in labels.iterrows():
    folder = f"../data/frames/{row['division']}/{row['division']}_{row['id']}"
    for img_path in Path(folder).glob("*.jpg"):
        image_paths.append(str(img_path))
        pattern_labels.append(row['pattern'])
        division_labels.append(row['division'])

# Load image
img = Image.open(image_paths[0]).convert("RGB")
display(img)

**Build Model**

In [ ]:
def build_resnet_model(num_classes: int):
    # model = models.resnet18(pretrained=True)
    # in_features = model.fc.in_features
    # model.fc = nn.Linear(in_features, num_classes)
    # return model
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    return model

**Data Augmentation**

In [ ]:
def build_transforms():
    return transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

Encode Labels

In [ ]:
patterns = sorted(set(pattern_labels))
label2id = {p:i for i,p in enumerate(patterns)}
y = np.array([label2id[p] for p in pattern_labels], dtype=np.int64)

class FrameDataset(Dataset):
    def __init__(self, paths, labels, tfm):
        self.paths = paths
        self.labels = labels
        self.tfm = tfm
    def __len__(self):
        return len(self.paths)
    def __getitem__(self, i):
        path = self.paths[i]               
        img = Image.open(path).convert("RGB")
        return self.tfm(img), self.labels[i]
tfm = build_transforms()
full_dataset = FrameDataset(image_paths, y, tfm)

# train and test loaders and splits
g = torch.Generator().manual_seed(42)
n = len(full_dataset)
n_train = int(0.8*n)
n_test = n - n_train
train_ds, test_ds = random_split(full_dataset, [n_train, n_test], generator=g)

pin = (device.type == "cuda")  # pin_memory only helps on CUDA
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,
                          num_workers=0, pin_memory=pin, persistent_workers=False)
test_loader   = DataLoader(test_ds,   batch_size=32, shuffle=False,
                          num_workers=0, pin_memory=pin, persistent_workers=False)

len(train_ds), len(test_ds), len(patterns)



Loss 

In [ ]:
num_classes = len(patterns)
model = build_resnet_model(num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

Train and Test Loop

In [ ]:
def run_epoch(loader, train=True):
    model.train(train)
    total_loss, correct, total = 0.0, 0, 0

    for xb, yb in loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        if train:
            optimizer.zero_grad()

        logits = model(xb)
        loss = criterion(logits, yb)

        if train:
            loss.backward()
            optimizer.step()

        total_loss += loss.item() * yb.size(0)
        pred = logits.argmax(1)
        correct += (pred == yb).sum().item()
        total += yb.size(0)

    return total_loss / max(total,1), correct / max(total,1)

EPOCHS = 5
for epoch in range(1, EPOCHS+1):
    train_loss, train_acc = run_epoch(train_loader, train=True)
    test_loss, test_acc = run_epoch(test_loader,   train=False)
    print(f"Epoch {epoch:02d} | "
          f"train loss {train_loss:.4f} acc {train_acc:.3f} | "
          f"val loss {test_loss:.4f} acc {test_acc:.3f}")


Save Model

In [ ]:
Path("models").mkdir(parents=True, exist_ok=True)
torch.save(
    {"state_dict": model.state_dict(),
    "label2id":label2id,
    "id2label":{v:k for k,v in label2id.items()}},
    "models/resnet_pattern_classifier.pth"
)
print("Saved model to models/resnet18_pattern.pt")


In [ ]:
checkpoint_path = "models/resnet_pattern_classifier.pth"
checkpoint = torch.load(checkpoint_path, map_location=device)   

num_classes = len(ckpt["label2id"])
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(checkpoint["state_dict"])           
model = model.to(device).eval()

id2label = {v:k for k,v in checkpoint["label2id"].items()}
print("Loaded classes:", id2label)


In [ ]:
tfm = build_transforms()
test_img_path = image_paths[0]  
img = Image.open(test_img_path).convert("RGB")
x = tfm(img).unsqueeze(0).to(device)

with torch.no_grad():
    logits = model(x)
pred_id = logits.argmax(1).item()
print("Prediction:", id2label[pred_id])
